In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons.optimizers.RectifiedAdam as RAdam
from tensorflow_addons.Lookahead import Lookahead

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# import dtale
# import seaborn as sns
# dtale.show(data_df, ignore_duplicate=True)

ModuleNotFoundError: No module named 'tensorflow_addons.optimizers.RectifiedAdam'

## 1. Data Exploration

In [ ]:
class DataInit():
    def __init__(self):
        self.path = './vehicles.csv'

    def get_df(self):
        return pd.read_csv(self.path)

    # Create a func for question 1 "Which manufacturer produces the most fuel efficient fleet of cars?"
    # Assumption is that higher combined MPG, more efficient the vehicles are.
    # def rank_manufacturers_by_fuel_efficiency(self, df):
    #     df = df[['make', 'comb08', 'combA08', 'atvType']]

    #     def fuel_efficiency_per_manufacturer(df):
    #         # Here Oliver makes an assumption that dual fuel vehicle only uses fuelType2 to be fuel efficient.
    #         df['comb_mpg'] = df.apply(lambda x: x['comb08'] if x['combA08'] == 0 else x['combA08'], axis=1)
    #         avg_barrel = df['comb_mpg'].mean()
    #         return avg_barrel

    #     series = df.groupby('make').apply(fuel_efficiency_per_manufacturer)
    #     series = series.sort_values(ascending=False)
    #     return series

In [ ]:
# Export descriptions of all columns for examination
data = DataInit()
data_df = data.get_df()

In [ ]:
# data_df.describe(include='all')

In [ ]:
# data_df.isnull().sum()

In [ ]:
# data_df.nunique()

## 2. Relationship Analysis

In [ ]:
correlation = data_df.corr() 
correlation['UCity'].head()

In [ ]:
# correlation

In [ ]:
fig1, ax1 = plt.subplots(figsize=(100, 30), dpi=50)
cor_plt = sns.heatmap(correlation[['UCity']], xticklabels=['UCity'], yticklabels=correlation.columns, annot=True, ax=ax1) # heatmap只是用颜色表示了数值而已，装逼用的
plt.show()

In [ ]:
cor_plt.figure.savefig('Q4_corr_heatmap.png')

## 3. Data Preparation

In [ ]:
# Numeric features normalization
def zscore_normalization(series):
    scaler = StandardScaler()
    transformed_num = scaler.fit_transform(np.expand_dims(series, axis=1))
    transformed_num = np.squeeze(transformed_num, axis=1)
    return transformed_num


def minmax_normalization(series):
    scaler = MinMaxScaler()
    transformed_num = scaler.fit_transform(np.expand_dims(series, axis=1))
    transformed_num = np.squeeze(transformed_num, axis=1)
    return transformed_num

# Fill missing values
def fill_missing_for_numeric_cols(x):
    mean = x.mean()
    x = x.fillna(mean)
    return x

In [ ]:
def tokenizer(inputs): 
    id2idx = {id: index for index, id in enumerate(inputs)}
    idx2id = {index: id for index, id in enumerate(inputs)}
    return id2idx, idx2id

In [ ]:
def get_trainable_data(df, numeric_cols):
    # Normalize numeric cols
    df[numeric_cols[:-1]] = df[numeric_cols[:-1]].apply(zscore_normalization) 
    df[numeric_cols[-1]] = df[numeric_cols[-1]].transform(minmax_normalization) 
    # fill in NaN for numeric cols
    df[numeric_cols] = df[numeric_cols].apply(fill_missing_for_numeric_cols) 
    # fill in NaN for atvType col 
    df = df.fillna(value={'atvType': 'Petrol'})

    atvtype2idx, idx2atvtype = tokenizer(df['atvType'].unique()) 
    engid2idx, idx2engid = tokenizer(df['engId'].unique())
    make2idx, idx2make = tokenizer(df['make'].unique())
    drive2idx, idx2drive = tokenizer(df['drive'].unique())

    # replace categorical cols (ids) with index
    df['atvType'] = df['atvType'].replace(atvtype2idx) 
    df['engId'].replace(engid2idx, inplace=True) 
    df['make'].replace(make2idx, inplace=True)
    df['drive'].replace(drive2idx, inplace=True)

    # get training data for RegressionEmbed model
    x_all_RE = df.values[:, :-1] # 
    y_all_RE = df.values[:, -1]
    # train/test/valid split 
    x_train_all_RE, x_test_RE, y_train_all_RE, y_test_RE = train_test_split(
        x_all_RE, y_all_RE, train_size=0.8, random_state=42)
    x_train_RE, x_valid_RE, y_train_RE, y_valid_RE = train_test_split(
        x_train_all_RE, y_train_all_RE, train_size=0.8, random_state=42)

    # get training data for LinerRegression
    x_all_num_lr = df.values[:, :5]
    x_all_cat_lr = df.values[:, 5:9]
    y_all_lr = df.values[:, -1]

    # One-Hot for categorical cols
    one_hot_encoder = OneHotEncoder(drop='first', categories='auto')
    x_all_cat_lr = one_hot_encoder.fit_transform(x_all_cat_lr).toarray()
    x_all_lr = np.hstack((x_all_num_lr, x_all_cat_lr))
    # train/test/valid split 
    x_train_all_lr, x_test_lr, y_train_all_lr, y_test_lr = train_test_split(
        x_all_lr, y_all_lr, train_size=0.8, random_state=42)
    x_train_lr, x_valid_lr, y_train_lr, y_valid_lr = train_test_split(
        x_train_all_lr, y_train_all_lr, train_size=0.8, random_state=42)

    return x_train_RE, y_train_RE, x_test_RE, y_test_RE, x_valid_RE, y_valid_RE, \
           atvtype2idx, idx2atvtype, engid2idx, idx2engid, make2idx, idx2make, drive2idx, idx2drive, \
           x_train_lr, y_train_lr, x_test_lr, y_test_lr, x_valid_lr, y_valid_lr


In [ ]:
def rmse(y_actual, y_pred):
    return np.sqrt(np.mean((y_actual - y_pred) ** 2))

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 600)
    plt.show()

In [ ]:
def get_embeddings_dataframe(model, layer_name, idx_dic):
    embedding_learnt = model.get_layer(name=layer_name).get_weights()[0]
    df = pd.DataFrame(embedding_learnt)
    df = df.rename(index=idx_dic)

    return df

In [ ]:
def get_weights_dataframe(model, layer_names):
    df = pd.DataFrame()
    index_dict = {index: name for index, name in enumerate(layer_names)}
    for idx, layer_name in enumerate(layer_names):
        if idx == 0:
            weights = model.get_layer(name=layer_name).get_weights()[0]
            df = pd.DataFrame(weights.T)
        else:
            weights = model.get_layer(name=layer_name).get_weights()[0]
            df = pd.concat([df, pd.DataFrame(weights.T)])
            index_dict[idx] = layer_name
    df = df.reset_index(drop=True)
    df = df.rename(index=index_dict)
    return df, index_dict

In [ ]:
class RegressionWithEmbed(keras.Model):
    def __init__(self):
        super(RegressionWithEmbed, self).__init__()

        self.numeric_weights = keras.layers.Dense(1, activation='relu', name='numericWeights')
        self.atvtype_embedding = keras.layers.Embedding(atvtype_count, EMBEDDING_DIM,
                                                        embeddings_initializer='uniform',
                                                        embeddings_regularizer=keras.regularizers.l2(
                                                            l=REGULIZATION_RATE),
                                                        name='atvtypeEmbed')
        self.atvtype_weights = keras.layers.Dense(1, activation='relu', name='atvtypeWeights')

        self.engid_embedding = keras.layers.Embedding(engid_count, EMBEDDING_DIM,
                                                      embeddings_initializer='uniform',
                                                      embeddings_regularizer=keras.regularizers.l2(
                                                          l=REGULIZATION_RATE),
                                                      name='engidEmbed')
        self.engid_weights = keras.layers.Dense(1, activation='relu', name='engidWeights')

        self.make_embedding = keras.layers.Embedding(make_count, EMBEDDING_DIM,
                                                     embeddings_initializer='uniform',
                                                     embeddings_regularizer=keras.regularizers.l2(
                                                         l=REGULIZATION_RATE),
                                                     name='makeEmbed')
        self.make_weights = keras.layers.Dense(1, activation='relu', name='makeWeights')

        self.drive_embedding = keras.layers.Embedding(drive_count, EMBEDDING_DIM,
                                                      embeddings_initializer='uniform',
                                                      embeddings_regularizer=keras.regularizers.l2(
                                                          l=REGULIZATION_RATE),
                                                      name='driveEmbed')
        self.drive_weights = keras.layers.Dense(1, activation='relu', name='driveWeights')

    def call(self, x):
        numeric_output = self.numeric_weights(x[0])
        atvtype_output = self.atvtype_weights(self.atvtype_embedding(x[1]))
        engid_output = self.engid_weights(self.engid_embedding(x[2]))
        make_output = self.make_weights(self.make_embedding(x[3]))
        drive_output = self.drive_weights(self.drive_embedding(x[4]))

        output = keras.layers.Add()([numeric_output,
                                     atvtype_output, engid_output, make_output, drive_output])

        return output

In [ ]:
# "Build a model to predict city mpg (variable “UCity” in column BG)."
# UCity is unadjusted city MPG for fuelType1

# STEP 1: Extract data from cvs and manually pick out features to use.
NUMERIC_COLS = ['barrels08',  # annual petroleum consumption in barrels for fuelType1 (1)
                'city08',  # city MPG for fuelType1 (2), (11)
                'displ',  # engine displacement in liters
                'highway08',  # highway MPG for fuelType1 (2), (11)
                'year'
                ]

CATEGORICAL_COLS = ['atvType',
                    'engId',
                    'make',
                    'drive'  # drive axle type
                    ]

TARGET = ['UCity']

data = DataInit()
data_df = data.get_df()
data_df = data_df[NUMERIC_COLS + CATEGORICAL_COLS + TARGET] 

# Explore features
print(data_df[NUMERIC_COLS].describe())
print()
for feature in CATEGORICAL_COLS:
    print("{} has {} unique values".format(feature, len(data_df[feature].unique())))

# STEP 2: Get training data for RegressionWithEmbed and LinearRegression
#         Because features "make" and "engID" have too many values (135/2661), one-hot encoding of LinearRegression might be problematic.
#         So RegressionWithEmbed is used, with LinearRegression as baseline.
x_train_RE, y_train_RE, x_test_RE, y_test_RE, x_valid_RE, y_valid_RE, \
atvtype2idx, idx2atvtype, engid2idx, idx2engid, make2idx, idx2make, drive2idx, idx2drive, \
x_train_lr, y_train_lr, x_test_lr, y_test_lr, x_valid_lr, y_valid_lr = get_trainable_data(data_df, numeric_cols=NUMERIC_COLS)

# STEP 3: Build and train RegressionWithEmbed
EPOCHS = 150
BATCH_SIZE = 32
BUFFER_SIZE = 10000
EMBEDDING_DIM = 6
LEARNING_RATE = 0.001
REGULIZATION_RATE = 0.00015

atvtype_count = len(atvtype2idx)
engid_count = len(engid2idx)
make_count = len(make2idx)
drive_count = len(drive2idx)

# RwE model initialization
RegressionWithEmbed = RegressionWithEmbed()

# optimizer and loss function and callbacks
opt = Lookahead(RAdam(LEARNING_RATE))
RegressionWithEmbed.compile(loss='mean_squared_error', optimizer=opt)
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)]

# train the model
RegressionWithEmbed_history = RegressionWithEmbed.fit(
    (x_train_RE[:, :5], x_train_RE[:, 5], x_train_RE[:, 6], x_train_RE[:, 7], x_train_RE[:, 8]), y_train_RE,
    validation_data=((x_valid_RE[:, :5], x_valid_RE[:, 5], x_valid_RE[:, 6], x_valid_RE[:, 7], x_valid_RE[:, 8]), y_valid_RE),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks
    )

# plot learning curve
plot_learning_curves(RegressionWithEmbed_history)
# evaluate the model
x = RegressionWithEmbed.evaluate(
    (x_test_RE[:, :5], x_test_RE[:, 5], x_test_RE[:, 6], x_test_RE[:, 7], x_test_RE[:, 8]), y_test_RE,
    batch_size=BATCH_SIZE)
print(x)

# STEP 4: Train LR
lr = LinearRegression()
lr.fit(x_train_lr, y_train_lr)

# STEP 5: Compare RwE and LR
# LR prediction
Y_pred_lr = lr.predict(x_test_lr)
# RwE prediction
Y_pred_RE = RegressionWithEmbed.predict(
    (x_test_RE[:, :5], x_test_RE[:, 5], x_test_RE[:, 6], x_test_RE[:, 7], x_test_RE[:, 8]))
Y_pred_RE = np.squeeze(Y_pred_RE)

test_dataset_results = pd.DataFrame(np.vstack((y_test_lr, Y_pred_lr, Y_pred_RE)).T, columns=['actual', 'lr', 'RE'])

print(test_dataset_results.describe())
print()

test_dataset_results.to_csv('test_dataset_results.csv')

print("lr's R2 is {}".format(r2_score(y_test_lr, Y_pred_lr)))
print("RE's R2 is {}".format(r2_score(y_test_RE, Y_pred_RE)))
print("lr's rmse is {}".format(rmse(y_test_lr, Y_pred_lr)))
print("RE's rmse is {}".format(rmse(y_test_RE, Y_pred_RE)))

'''As shown above, because some one-hot encoding features have extremely large weights, lr prediction fails at predicting some un-seen type of test data'''
'''But RwW model converts categorical features into short fixed-length embeddings, hence it doesn't have above problem.'''

# Check LR's performance after filtering out extremely wrong cases.
print('LR has {} extreme predictions out of {} test samples'.format(
    len(test_dataset_results[test_dataset_results['lr'] > 10000]) + len(test_dataset_results[test_dataset_results['lr'] < -10000]), len(test_dataset_results)))
print()
filtered_test_dataset_results = test_dataset_results[
    ~((test_dataset_results['lr'] > 10000) | (test_dataset_results['lr'] < -10000))]

print("After filter out extremely wrong predictions of lr, lr's R2 is {}".format(
    r2_score(filtered_test_dataset_results['lr'].values, filtered_test_dataset_results['actual'].values)))
print("After filter out extremely wrong predictions of lr, lr's rmse is {}".format(
    rmse(filtered_test_dataset_results['lr'].values, filtered_test_dataset_results['actual'].values)))

# Evaluate the weights of LR
np.savetxt("lr_coef_.csv", lr.coef_, delimiter=",")



In [ ]:
# Evaluate the weights of RwE
numeric_layer_name = ['numericWeights']
categorical_layer_names = ['atvtypeWeights', 'engidWeights', 'makeWeights', 'driveWeights']

numeric_weights, _ = get_weights_dataframe(RegressionWithEmbed, numeric_layer_name)
categorical_weights, _ = get_weights_dataframe(RegressionWithEmbed, categorical_layer_names)

numeric_weights.to_csv('RwE_numeric_weights.csv')
categorical_weights.to_csv('RwE_categorical_weights.csv')

print(numeric_layer_name)
print()
print(categorical_weights)

# Export the embeddings matrix for all categorical weights
atvtype_embedding_df= get_embeddings_dataframe(RegressionWithEmbed, 'atvtypeEmbed', idx2atvtype)
engid_embedding_df = get_embeddings_dataframe(RegressionWithEmbed, 'engidEmbed', idx2engid)
make_embedding_df = get_embeddings_dataframe(RegressionWithEmbed, 'makeEmbed', idx2make)
drive_embedding_df = get_embeddings_dataframe(RegressionWithEmbed, 'driveEmbed', idx2drive)

atvtype_embedding_df.to_csv('atvtype_embedding.csv')
engid_embedding_df.to_csv('engid_embedding.csv')
make_embedding_df.to_csv('make_embedding.csv')
drive_embedding_df.to_csv('drive_embedding.csv')